### Connect to your workspace


In [ ]:
conda env create -f myenv.yml

In [1]:
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Ready to use Azure ML 1.36.0


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

azure_ml loaded


### Create and register the dataset

In [ ]:
# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

In [ ]:
default_ds.upload_files(files=['./data/diamonds.csv'],
                       target_path='diamond-data/', 
                       overwrite=True,
                       show_progress=True)

In [ ]:
from azureml.core import Dataset

# Get the default datastore
default_ds = ws.get_default_datastore()

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diamond-data/*.csv'))

# Display the first 5 rows as a Pandas dataframe
tab_data_set.take(5).to_pandas_dataframe()

In [ ]:
# Register the tabular dataset
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                                        name='diamond dataset',
                                        description='diamond data',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)
    
print('Datasets registered')

### Create Environnement

In [4]:
from azureml.core import Environment

tf_env = Environment.from_existing_conda_environment(name="tf_env", conda_environment_name="az_env")
tf_env.save_to_directory(path="./tf_env", overwrite=True)

Exporting conda specifications for existing conda environment: az_env


In [5]:
from azureml.core import Experiment

experiment_name = 'dimond_experiment'
experiment = Experiment(workspace=ws, name=experiment_name)

### Create and run experiment

In [19]:
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails

# Get the training dataset
diamond_ds = ws.datasets.get("diamond dataset")

src = ScriptRunConfig(source_directory='./training',
                      script='model.py',
                      compute_target='local',
                      arguments = ['--input-data', diamond_ds.as_named_input('training_data')],
                      environment=tf_env)

run = experiment.submit(config=src)

# Show the running experiment run in the notebook widget
RunDetails(run).show()

# Block until the experiment run has completed
run.wait_for_completion()

In [20]:
run = experiment.submit(config=src)

# Show the running experiment run in the notebook widget
RunDetails(run).show()

# Block until the experiment run has completed
run.wait_for_completion()

NameError: name 'RunDetails' is not defined

In [8]:
# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

RMSE 884.2755126953125
MAPE 13.783226013183594


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/1700_azureml.log
logs/azureml/dataprep/backgroundProcess.log
logs/azureml/dataprep/backgroundProcess_Telemetry.log
outputs/keras_metadata.pb
outputs/saved_model.pb
outputs/variables/variables.data-00000-of-00001
outputs/variables/variables.index


### Register the saved model

In [18]:
from azureml.core.model import Model
from azureml.core.resource_configuration import ResourceConfiguration


model = run.register_model(model_path='./outputs', 
                        model_name='diamond_price_predictor',
                        model_framework=Model.Framework.TENSORFLOW,
                        model_framework_version='2.6',
                        resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                        tags={'Training context':'Script'},
                        properties={'RMSE': run.get_metrics()['RMSE'], 'MAPE': run.get_metrics()['MAPE']})

### Deploy the model

In [13]:
service = Model.deploy(ws, "tensorflow-web-service", [model])

In [14]:
print(service.scoring_uri)

None
